
RusVectōrēs: семантические модели для русского языка
Елизавета Кузьменко, Андрей Кутузов
https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb

https://vectors.nlpl.eu/repository/20/180.zip
    "dimensions": 300,
    "vocabulary size": 189193,
189193 300
так_ADV 1.0967804 -2.2944486 1.97916785428456 -1.2069672
быть_VERB 0.56827927 -0.3209115 -0.019124394 -1.9356475 
мочь_VERB 2.639508 -3.596937 -0.13203041 -0.4291383 0.9506081 


In [39]:
#pip install pymystem3 requests

import sys
import gensim, logging
import numpy as np

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#import zipfile
#model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
#m = wget.download(model_url)
#model_file = model_url.split('/')[-1]
#model_file = "180.zip"

"""

NB


Из zip-архива у меня не заработало.
Тогда из файла 'http://vectors.nlpl.eu/repository/11/180.zip' 
вручную выложил в директорию '180_model.bin'


"""

extracted_model_file = '180_model.bin'

#with zipfile.ZipFile(model_file, 'r') as archive:
#Extract the model.bin file to the current directory
#archive.extract('model.bin', path='.')

# Now load the extracted model file
model = gensim.models.KeyedVectors.load_word2vec_format(extracted_model_file, binary=True)

2025-01-21 16:46:11,640 : INFO : loading projection weights from 180_model.bin
2025-01-21 16:46:15,265 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (189193, 300) matrix of type float32 from 180_model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-01-21T16:46:15.265992', 'gensim': '4.3.3', 'python': '3.8.20 | packaged by conda-forge | (default, Sep 30 2024, 17:44:03) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}


In [40]:
import requests
from pymystem3 import Mystem

def tag_mystem(text="Текст нужно передать функции в виде строки!", mapping=None, postags=True):
    processed = m.analyze(text)
    #https://github.com/nlpub/pymystem3/blob/master/pymystem3/mystem.py
    tagged = []
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы
   
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]
                else:
                    pos = "X"
                    
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue
            
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    
    return tagged

# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
#mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"
#mystem2upos = {}
#r = requests.get(mapping_url, stream=True)

#for pair in r.text.split("\n"):
#    pair = pair.split()
#    if len(pair) > 1:
#        mystem2upos[pair[0]] = pair[1]
# Захардкодим таблицу преобразования частеречных тегов Mystem в теги UPoS
mystem2upos = {
    "A": "ADJ",
    "ADV": "ADV",
    "ADVPRO": "ADV",
    "ANUM": "ADJ",
    "APRO": "DET",
    "COM": "ADJ",
    "CONJ": "SCONJ",
    "INTJ": "INTJ",
    "NONLEX": "X",
    "NUM": "NUM",
    "PART": "PART",
    "PR": "ADP",
    "S": "NOUN",
    "SPRO": "PRON",
    "UNKN": "X",
    "V": "VERB"
}
print("Loading the tokenize_lemmatize_taggize model...")
m = Mystem()


Loading the tokenize_lemmatize_taggize model...


In [41]:
m = Mystem()
text = "стекло"
analysis = m.analyze(text)
print(analysis)
for item in analysis:
    if 'analysis' in item:
        print(f"Слово: {item['text']}")
        for variant in item['analysis']:
            print(f"  Лемма: {variant['lex']}, Грамм. информация: {variant['gr']}")
    else:
        print(f"Пробел или пунктуация: {item['text']}")


[{'analysis': [{'lex': 'стекло', 'wt': 0.9853860572, 'gr': 'S,сред,неод=(вин,ед|им,ед)'}], 'text': 'стекло'}, {'text': '\n'}]
Слово: стекло
  Лемма: стекло, Грамм. информация: S,сред,неод=(вин,ед|им,ед)
Пробел или пунктуация: 



In [42]:
#не более 10 слов через пробел. RusVectōrēs сложит вектора положительных слов и вычтет из них 
#отрицательные. Затем он выдаст слова, наиболее близкие к получившемуся вектору. Если вы оставите
# отрицательное поле пустым, RusVectōrēs просто найдет центр лексического кластера, образованного 
# положительными словами.
def tokenize_lemmatize_taggize(input_text):
    output = tag_mystem(text=input_text, mapping=mystem2upos)
    return " ".join(output)

#result = tokenize_lemmatize_taggize("россией священное наши державы") print("test: " , result)


def get_custom_similar_words(model, num_otvetov, positive_str, negative_str):
    # Обработка строк и добавление суффикса _NOUN
    positive_words = [f"{word}_NOUN" for word in positive_str.split()]
    negative_words = [f"{word}_NOUN" for word in negative_str.split()]

    positive_words = [word for word in tokenize_lemmatize_taggize(positive_str).split() if word in model]
    negative_words = [word for word in tokenize_lemmatize_taggize(negative_str).split() if word in model]

    print("++: ", positive_words, "\n--: " , negative_words)
    if not positive_words:
         return
    
    ret = np.array([], dtype=str)#    ret = np.unique([], dtype=str)
    
    # Получаем наиболее похожие слова
    for i in model.most_similar(positive=positive_words, negative=negative_words, topn=num_otvetov):
            # слово + коэффициент косинусной близости print(i[0], i[1])
            processed_string = i[0].split("_")[0]
            if processed_string not in ret:
                ret = np.append(ret, processed_string)
            
    #ret = np.unique(ret)
    print(' '.join(ret))
    return


In [55]:
get_custom_similar_words(model, 99,
                       "чугун расплав мастер"
                     #   "трезвость опыт" # <-- это вычитается
                    , ""
                         )

++:  ['чугун_NOUN', 'расплав_NOUN', 'мастер_NOUN'] 
--:  []
огнеупор литье изложница тиголь плавка отливка шихта огнеупорный жаропрочный металл обжиг фурма формовка кристаллизатор шихвать расплава кокс высокопрочный поковка пресс-форма выплавка обжига ковкий мартеновский сталь конвертер формовочный конвертор конвертерный олово плавильный сплаво выплавлять спекание стекломасса карбид битум каолин прокатка легировать литейщик обжигй прессование тугоплавкий электропечь медь низколегированный нержаветь клинкер штамповка цирконий расплавить формовать разливка асбест ниобие монокристалл высокотемпературный электролизер бессемеровский домна легкоплавкий дутье графитовый латунь иттрий ванадий электродуговый титановый вольфрам стеклопластик окалина кокиль заполнитель вагранка марганцевый нержавеющий ковка расплавлять электролитический глинозем сплав шлак сталеплавильный мартен формование болванка электролиз пруток доменный термический ферросплав термообработка литый кокильный футеровка упрочнен

In [54]:
# Определяем два массива слов
first_array = "лошадь тропа куст козуль охотник двустволка гончий путик берданка заяц куропатка пес тропка егерь кабан винтовка дробовик опушка коза легавый засадка олень изюбр чаща палка кабарга волк зверовый ружье дичь овраг борзая медведица загонщик лось лес ягдташа гончая рябчик сохатый ездовый кочка сторожка речка рогатина зверь валежник козуля просека тетеревь лесок водопой лайка шалаш охотничий карабин пороша борзятник бекас стая глухарь лощина росомаха зимник собачонка собака изгородь нарты косуля лощинка нарт лисица нарта полверст медведь вальдшнеп косач болотец тетеревенка выводка стойбище дрофа полянка дорожка чжан-бао тетерево сарай сеттер одностволка дупель утка ружьишко косогор"
second_array = "куст тропка тропа палка дорожка лошадь забор винтовка калитка изгородь жердь пес овраг коновязь плетень сарай просека колышек сторожка поводок косогор полверст дорога веревка сзади карабин берданка загородка опушка лощинка собачонка караульщик бревн мосток хворостина пригорок канава цепь нарта заборчик колышка жердина лощина тропочка гуськом нога путик шалаш мостки конь ворота чаща полянка ограда двустволка сакля загородь будка хворостинка насыпь повозка коза уздечка гать овчарка двор лесин речка крыльцо след улица спешиться лыжа телега марунич лаз караулок спина нарты лесок рогатина ездовый прут канавка развилина стежка сворка шест ствол бугор передка арба обрыв поляна лыжня прогалина"

# Преобразуем строки в списки
first_list = first_array.split()
second_list = second_array.split()

# Находим слова, которые есть в первом массиве, но отсутствуют во втором
missing_words = set(first_list) - set(second_list)

# Выводим результат
print("Слова, которые были в первом массиве, но отсутствуют во втором:")
for word in missing_words:
    print(word)

Слова, которые были в первом массиве, но отсутствуют во втором:
козуля
чжан-бао
гончий
лайка
тетерево
бекас
медведица
волк
кабарга
болотец
ружье
лисица
кабан
лес
валежник
ягдташа
сеттер
легавый
борзая
стойбище
засадка
изюбр
дрофа
заяц
стая
медведь
собака
кочка
охотник
тетеревь
егерь
тетеревенка
козуль
нарт
зимник
косач
загонщик
утка
дичь
росомаха
одностволка
дробовик
охотничий
зверовый
лось
дупель
вальдшнеп
рябчик
глухарь
куропатка
косуля
сохатый
гончая
ружьишко
пороша
олень
выводка
борзятник
водопой
зверь
